In [1]:
import numpy as np
import cv2
import torch
import torchvision.models as models

%load_ext autoreload
%autoreload 2


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## pretrained embedder

In [3]:
checkpoint = torch.load('A3/best_model.pth')
model = models.resnet50(weights=checkpoint)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 136)
model.load_state_dict(checkpoint)

feature_extractor = torch.nn.Sequential(*list(model.children())[:-3])
feature_extractor.to(device)
feature_extractor.eval()
feature_extractor.compile()

/home/lanv/Desktop/Faks/mag/SB/assignment_3/venv/lib64/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## dataloading

In [4]:
from A3.dataloader import create_dataloaders
import os
os.chdir(os.getcwd())

train_dir = os.path.join("A3",'datasets', 'ears', 'images-cropped', 'train')
test_dir = os.path.join("A3",'datasets', 'ears', 'images-cropped', 'test') # ?? 
val_dir = os.path.join("A3", 'datasets', 'ears', 'images-cropped', 'val') # ?? 
train_loader, val_loader, test_loader, num_classes = create_dataloaders(train_dir, test_dir, val_dir, 8)

len images =  2723
len images =  414


In [11]:
# sample = next(iter(test_loader))
# test = torch.tensor(sample[0], dtype=torch.float).to(device)
# feature_extractor(test)

## Embed ears

Ear embeddings will be checked against the know ears to determine identity

- Using all the images in the test set compute and store these ResNet feature vectors and compute and store the LBP feature vectors.

In [6]:
labels = []
features = []

# from torch.cuda import ipc_collect

for image, labels_arr in iter(test_loader):
  labels.append(labels_arr)
  image_embeddings = feature_extractor(image.to(device))
  # features.append(image_embeddings.detach().cpu().numpy())
  # ipc_collect()
  # torch.cuda.empty_cache()


/tmp/tmpuqc7skxw/main.c:4:10: fatal error: Python.h: No such file or directory
    4 | #include <Python.h>
      |          ^~~~~~~~~~
compilation terminated.
/tmp/tmpzevr92v3/main.c:4:10: fatal error: Python.h: No such file or directory
    4 | #include <Python.h>
      |          ^~~~~~~~~~
/tmp/tmpau8ajlem/main.c:4:10: fatal error: Python.h: No such file or directory
    4 | #include <Python.h>
      |          ^~~~~~~~~~
compilation terminated.
compilation terminated.
/tmp/tmpn9ie6wh_/main.c:4:10: fatal error: Python.h: No such file or directory
    4 | #include <Python.h>
      |          ^~~~~~~~~~
compilation terminated.
/tmp/tmp2_08lkr7/main.c:4:10: fatal error: Python.h: No such file or directory
    4 | #include <Python.h>
      |          ^~~~~~~~~~
compilation terminated.
/tmp/tmpt3laz65m/main.c:4:10: fatal error: Python.h: No such file or directory
    4 | #include <Python.h>
      |          ^~~~~~~~~~
compilation terminated.
/tmp/tmpnjgz_xjn/main.c:4:10: fatal error: Pyt

BackendCompilerFailed: backend='inductor' raised:
CalledProcessError: Command '['/usr/bin/gcc', '/tmp/tmpxjv63x5n/main.c', '-O3', '-I/home/lanv/Desktop/Faks/mag/SB/assignment_3/venv/lib64/python3.11/site-packages/triton/common/../third_party/cuda/include', '-I/usr/include/python3.11', '-I/tmp/tmpxjv63x5n', '-shared', '-fPIC', '-lcuda', '-o', '/tmp/tmpxjv63x5n/triton_.cpython-311-x86_64-linux-gnu.so', '-L/lib64', '-L/lib64']' returned non-zero exit status 1.

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True
